In [1]:
import numpy as np

In [2]:
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as ply
import os
import imageio
from PIL import Image
import glob

% matplotlib inline

In [3]:
def rle (img):
    flat_img = img.flatten()
    flat_img = np.where(flat_img > 0.5, 1, 0).astype(np.uint8)

    starts = np.array((flat_img[:-1] == 0) & (flat_img[1:] == 1))
    ends = np.array((flat_img[:-1] == 1) & (flat_img[1:] == 0))
    starts_ix = np.where(starts)[0] + 2
    ends_ix = np.where(ends)[0] + 2
    lengths = ends_ix - starts_ix
    
    return starts_ix, lengths

In [12]:
img = np.array(imageio.imread('./data/train/images/0ba541766e.png'), dtype=np.uint8)

In [13]:
img_mask = np.array(imageio.imread('./data/train/masks/0ba541766e.png'), dtype=np.uint8)

In [14]:
img_mask

array([[255, 255, 255, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0],
       ...,
       [255, 255, 255, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0]], dtype=uint8)

In [15]:
img_mask.sum()

1141125

In [17]:
import pandas as pd

In [18]:
df_train = pd.read_csv('./data/train.csv')

In [20]:
df_train.loc[df_train.id=='0ba541766e']

,id,rle_mask
1483,0ba541766e,1 3936 3940 90 4041 81 4142 72 4243 62 4344 46...


In [22]:
img_mask.shape

(101, 101)

In [24]:
img_mask.clip(0,1)

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=uint8)

In [39]:
def rlencode(x, dropna=False):
    """
    Run length encoding.
    Based on http://stackoverflow.com/a/32681075, which is based on the rle 
    function from R.
    
    Parameters
    ----------
    x : 1D array_like
        Input array to encode
    dropna: bool, optional
        Drop all runs of NaNs.
    
    Returns
    -------
    start positions, run lengths, run values
    
    """
    where = np.flatnonzero
    x = np.asarray(x)
    n = len(x)
    if n == 0:
        return (np.array([], dtype=int), 
                np.array([], dtype=int), 
                np.array([], dtype=x.dtype))

    starts = np.r_[0, where(~np.isclose(x[1:], x[:-1], equal_nan=True)) + 1]
    lengths = np.diff(np.r_[starts, n])
    values = x[starts]
    
    if dropna:
        mask = ~np.isnan(values)
        starts, lengths, values = starts[mask], lengths[mask], values[mask]
    
    return starts[::2]+1, lengths[::2]

In [44]:
img_mask.shape

(101, 101)

In [40]:
rlencode(img_mask.T.flatten())

(array([   1, 3940, 4041, 4142, 4243, 4344, 4445, 4546, 4647, 4748, 4849,
        4950, 5051], dtype=int64),
 array([3936,   90,   81,   72,   62,   46,   44,   38,   35,   28,   21,
          14,    8], dtype=int64))

In [43]:
df_train.loc[df_train.id=='0ba541766e'].rle_mask.iloc[0]

'1 3936 3940 90 4041 81 4142 72 4243 62 4344 46 4445 44 4546 38 4647 35 4748 28 4849 21 4950 14 5051 8'